<a href="https://colab.research.google.com/github/Fernando-LunaP/Matematicas/blob/main/Ejercicio_SVD_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Alumno:** Fernando Luna Ponce  
**Asignatura:** Matemáticas para para Ciencia de Datos


In [42]:
# generate related variables
import numpy as np
from numpy.random import randn
from matplotlib import pyplot
import matplotlib.pyplot as plt
from IPython import display
from sklearn.preprocessing import StandardScaler
import pandas as pd
import plotly.express as px
import statsmodels.api as sm

# Creación de matriz A_std a partir de dos variables

Vamos a tomar como como ejemplo el dataset USArrests que viene en la librería statsmodel que contiene el número de asaltos, asesinatos, violaciones por cada 100,000 habitantes, también incluye el porcentaje de población viviendo en áreas urbanas en cada uno de los Estados de USA.

In [43]:
USArrests = sm.datasets.get_rdataset("USArrests", "datasets")
X = USArrests.data
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, Alabama to Wyoming
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Murder    50 non-null     float64
 1   Assault   50 non-null     int64  
 2   UrbanPop  50 non-null     int64  
 3   Rape      50 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 2.0+ KB


In [44]:
X.shape

(50, 4)

Los aspectos principales cuando tenemos que hacer un PCA es analizar, la media y la varianza de cada una de sus columnas.

In [45]:
print('Media de cada variable')
X.mean(axis=0)

Media de cada variable


Murder        7.788
Assault     170.760
UrbanPop     65.540
Rape         21.232
dtype: float64

In [46]:
print('Varianza de cada variable')
X.var(axis=0)

Varianza de cada variable


Murder        18.970465
Assault     6945.165714
UrbanPop     209.518776
Rape          87.729159
dtype: float64

El siguiente paso es realizar una estandarización de los datos para que estos tengan media igual a 0 y varianza igual a 1

In [47]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)
for i in range(X_std.shape[1]):
  print('Variable: ', X.columns[i])
  print('Media: ', round(X_std[:, i].mean(), 4))
  print('Varianza: ', round(X_std[:, i].var(), 4))
  print('--------------------')

Variable:  Murder
Media:  -0.0
Varianza:  1.0
--------------------
Variable:  Assault
Media:  0.0
Varianza:  1.0
--------------------
Variable:  UrbanPop
Media:  -0.0
Varianza:  1.0
--------------------
Variable:  Rape
Media:  0.0
Varianza:  1.0
--------------------


Generamos nuestros dataframes

In [48]:
# Dataframe A
A = pd.DataFrame(X)
# Dataframe A_std
A_std = pd.DataFrame(X_std)
# Dataframe A_centered
A_centered = pd.DataFrame(X - X.mean())

Hacemos el scatterplot con las primeras dos columnas

In [49]:
import plotly.graph_objects as go
fig_1 = px.scatter(A, x=A.columns[0], y=A.columns[1],
                  opacity=0.8,
                   color_discrete_sequence=['blueviolet']
                  )
fig_2 = px.scatter(A_std, x=A_std.columns[0], y=A_std.columns[1], 
                    opacity=0.8,
                   color_discrete_sequence=['orange']
                   )
fig_3 = px.scatter(A_centered, x=A_centered.columns[0], y=A_centered.columns[1], 
                   opacity=0.8,
                   color_discrete_sequence=['gray']
                   )
fig = go.Figure(data=fig_1.data + fig_2.data + fig_3.data )
fig.show()

# Matriz de covarianza vs $A^T_{std}A_{std}$

Tenemos que la covarianza asociada a las columnas de $A_{std\_scaler}$ es: 
<center> 
$Cov(A_{std\_scaler}) = \frac{1}{n - 1}A^T_{std\_scaler}A_{std\_scaler}$
</center>

In [50]:
Matriz_cov = (1/(A.shape[0] - 1)) * A_std.T.dot(A_std)
Matriz_cov

,0,1,2,3
0,1.020408,0.818238,0.070992,0.575080
1,0.818238,1.020408,0.264155,0.678818
2,0.070992,0.264155,1.020408,0.419736
3,0.575080,0.678818,0.419736,1.020408


Obtenemos el mismo resultado calculándolo que utilizando la función de pandas para obtener la covarianza

In [51]:
Matriz_covarianza = A_std.cov()
Matriz_covarianza

,0,1,2,3
0,1.020408,0.818238,0.070992,0.575080
1,0.818238,1.020408,0.264155,0.678818
2,0.070992,0.264155,1.020408,0.419736
3,0.575080,0.678818,0.419736,1.020408


Sea $r$ el rango de la matriz $A_{std\_scaler}$, si consideramos una descomposición en valores singulares de la matriz $A_{std\_scaler}$, tenemos que existen $r$ autovectores $v_1, v_2,...v_r$ de $A^T_{std\_scaler}A_{std\_scaler}$ que también lo son de la matriz de de covarianza

In [52]:
r = np.linalg.matrix_rank(A_std) # Rango de A_std
r

4

In [53]:
from numpy.linalg import eig
auto_valores, auto_vectores = eig(A_std.T.dot(A_std))
print('Autovalores de la matriz A^T_stdA_std = \n', auto_valores)
print('Autovectores de la matriz A^T_stdA_std = \n', auto_vectores)

Autovalores de la matriz A^T_stdA_std = 
 [124.01207896  49.48825763   8.67150439  17.82815903]
Autovectores de la matriz A^T_stdA_std = 
 [[ 0.53589947  0.41818087  0.6492278  -0.34123273]
 [ 0.58318363  0.1879856  -0.74340748 -0.26814843]
 [ 0.27819087 -0.87280619  0.13387773 -0.37801579]
 [ 0.54343209 -0.16731864  0.08902432  0.81777791]]


In [54]:
auto_valores, auto_vectores = eig(Matriz_covarianza)
print('Autovalores de la matriz de covarianza de A_std = \n', auto_valores)
print('Autovectores de la matriz de covarianza de A_std = \n', auto_vectores)

Autovalores de la matriz de covarianza de A_std = 
 [2.53085875 1.00996444 0.17696948 0.36383998]
Autovectores de la matriz de covarianza de A_std = 
 [[ 0.53589947  0.41818087  0.6492278  -0.34123273]
 [ 0.58318363  0.1879856  -0.74340748 -0.26814843]
 [ 0.27819087 -0.87280619  0.13387773 -0.37801579]
 [ 0.54343209 -0.16731864  0.08902432  0.81777791]]


Vemos que coinciden los autovalores.

In [55]:
auto_valores[0] * auto_vectores[0] - Matriz_covarianza.dot(auto_vectores[0])

0    0.617424
1    0.253286
2    0.975345
3   -1.379973
dtype: float64

In [56]:
auto_valores[1] * auto_vectores[1] - Matriz_covarianza.dot(auto_vectores[1])

0    0.047076
1   -0.100748
2    0.029257
3   -0.148150
dtype: float64

In [57]:
auto_valores[2] * auto_vectores[2] - Matriz_covarianza.dot(auto_vectores[2])

0    0.687411
1    0.729771
2    0.256556
3    0.695134
dtype: float64

In [58]:
auto_valores[3] * auto_vectores[3] - Matriz_covarianza.dot(auto_vectores[3])

0   -0.696502
1   -0.913439
2   -0.396083
3   -0.773232
dtype: float64

In [59]:
fig = px.scatter(A_std,
                 x=A_std.columns[0], 
                 y=A_std.columns[1], 
                 color_discrete_sequence=['orange'], 
                 marginal_x='violin', 
                 marginal_y='violin',
                 opacity=0.6
                )


arrow_1 = go.layout.Annotation(dict(
                x=  auto_valores[1] * auto_vectores[0][0],
                y=  auto_valores[1] * auto_vectores[0][1],
                xref="x", yref="y",
                text="",
                showarrow=True,
                axref = "x", ayref='y',
                ax= 0,
                ay= 0,
                arrowhead = 3,
                arrowwidth=2.5,
                arrowcolor=' mediumseagreen',)
            )

arrow_2 = go.layout.Annotation(dict(
                x=  auto_valores[0] * auto_vectores[1][0],
                y=  auto_valores[0] * auto_vectores[1][1],
                xref="x", yref="y",
                text="",
                showarrow=True,
                axref = "x", ayref='y',
                ax= 0,
                ay= 0,
                arrowhead = 3,
                arrowwidth=2.5,
                arrowcolor='blueviolet',)
            )


fig.update_layout(annotations= [arrow_1,arrow_2])
fig.show()
#fig.write_image("figura_matriz_covarianza.pdf", width=900, height=600)

<center>
$corr(x,y) = \frac{{\rm cov(x,y)}}{{\rm std}(x){\rm std}(y)}=\frac{\langle x_{std}, y_{std} \rangle}{n-1}$
</center>

In [60]:
S = A_std.T.dot(A_std) # La matriz de covarianza es S/(n-1)
S

,0,1,2,3
0,50.000000,40.093666,3.478631,28.178942
1,40.093666,50.000000,12.943585,33.262061
2,3.478631,12.943585,50.000000,20.567062
3,28.178942,33.262061,20.567062,50.000000


In [61]:
auto_valores, auto_vectores = eig(S)
print(auto_valores)
print(auto_vectores)

[124.01207896  49.48825763   8.67150439  17.82815903]
[[ 0.53589947  0.41818087  0.6492278  -0.34123273]
 [ 0.58318363  0.1879856  -0.74340748 -0.26814843]
 [ 0.27819087 -0.87280619  0.13387773 -0.37801579]
 [ 0.54343209 -0.16731864  0.08902432  0.81777791]]


Realizamos el cálculo de los valores singulares mediante la siguiente fórmula:
<center>
$\sigma_i = \sqrt{\lambda_{i}}$ para $i = {1,2...r}$
</center>

In [62]:
print('singular_value_1 =', np.sqrt(auto_valores[0]).round(4))
print('singular_value_2 =', np.sqrt(auto_valores[1]).round(4))
print('singular_value_3 =', np.sqrt(auto_valores[2]).round(4))
print('singular_value_4 =', np.sqrt(auto_valores[3]).round(4))

singular_value_1 = 11.1361
singular_value_2 = 7.0348
singular_value_3 = 2.9447
singular_value_4 = 4.2223


# Creación de componentes principales a través de SVD  
- Se aplica la SVD a A_std y se calculan las matrices U, Vt y un arreglo con los valores singulares $\sigma_{1}$ y $\sigma_{2}$
- U es una matriz con columnas $u_{1}$ y $u_{2}$
- Se hace el cálculo componentes:

componente_principal_i = $\sigma_{i}u_{i}$ (column vector) con $i= {1,2...r}$

Obtenemos los valores singulares

In [63]:
from scipy.linalg import svdvals
print('Valores singulares A_std = ',svdvals(A_std))

Valores singulares A_std =  [11.13607107  7.0347891   4.22234047  2.94474182]


Realizamo la descomposición en valores singulares

In [64]:
from scipy import linalg
U, S, Vt = linalg.svd(A_std, full_matrices=False)

In [65]:
U.shape, S.shape, Vt.shape

((50, 4), (4,), (4, 4))

In [66]:
print('Valores singulares de A_std = ', S)

Valores singulares de A_std =  [11.13607107  7.0347891   4.22234047  2.94474182]


In [67]:
Matriz_U = pd.DataFrame(U)
Matriz_Vt = pd.DataFrame(Vt)

Obtenemos las componentes principales

In [68]:
print('componente_principal_1 =\n ', S[0]*Matriz_U[0])
print('componente_principal_2 =\n ', S[1]*Matriz_U[1])
print('componente_principal_3 =\n ', S[2]*Matriz_U[2])
print('componente_principal_4 =\n ', S[3]*Matriz_U[3])

componente_principal_1 =
  0    -0.985566
1    -1.950138
2    -1.763164
3     0.141420
4    -2.523980
5    -1.514563
6     1.358647
7    -0.047709
8    -3.013042
9    -1.639283
10    0.912657
11    1.639800
12   -1.378911
13    0.505461
14    2.253646
15    0.796881
16    0.750859
17   -1.564818
18    2.396829
19   -1.763369
20    0.486166
21   -2.108441
22    1.692682
23   -0.996494
24   -0.696787
25    1.185452
26    1.265637
27   -2.874395
28    2.383915
29   -0.181566
30   -1.980024
31   -1.682577
32   -1.123379
33    2.992226
34    0.225965
35    0.311783
36   -0.059122
37    0.888416
38    0.863772
39   -1.320724
40    1.987775
41   -0.999742
42   -1.355138
43    0.550565
44    2.801412
45    0.096335
46    0.216903
47    2.108585
48    2.079714
49    0.629427
Name: 0, dtype: float64
componente_principal_2 =
  0     1.133392
1     1.073213
2    -0.745957
3     1.119797
4    -1.542934
5    -0.987555
6    -1.088928
7    -0.325359
8     0.039229
9     1.278942
10   -1.570460
11    0

Notar que las columas de $U$ tienen media cero pero los renglones de $V^T$ no necesariamente.

In [69]:
Matriz_U.describe().round(6)

,0,1,2,3
count,50.000000,50.000000,50.000000,50.000000
mean,-0.000000,0.000000,0.000000,-0.000000
std,0.142857,0.142857,0.142857,0.142857
min,-0.270566,-0.223242,-0.324452,-0.324097
25%,-0.123290,-0.103363,-0.102635,-0.071768
50%,0.016088,-0.021817,0.007390,-0.002041
75%,0.100327,0.109070,0.060766,0.072833
max,0.268697,0.340280,0.483145,0.365668


In [70]:
Matriz_Vt.T.describe().round(6)

,0,1,2,3
count,4.000000,4.000000,4.000000,4.000000
mean,-0.485177,-0.108490,-0.042405,0.032181
std,0.139541,0.563596,0.575270,0.576153
min,-0.583184,-0.872806,-0.378016,-0.743407
25%,-0.553370,-0.343691,-0.350428,-0.119084
50%,-0.539666,0.010333,-0.304691,0.111451
75%,-0.471472,0.245534,0.003333,0.262715
max,-0.278191,0.418181,0.817778,0.649228


# PCA
PCA se aplica a una matriz previamente estandarizada.

In [71]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(A_std)
principalDf = pd.DataFrame(data = principalComponents
             ,columns = ['componente_principal_1', 'componente_principal_2', 'componente_principal_3', 'componente_principal_4'])

In [72]:
principalDf

,componente_principal_1,componente_principal_2,componente_principal_3,componente_principal_4
0,0.985566,1.133392,-0.444269,0.156267
1,1.950138,1.073213,2.040003,-0.438583
2,1.763164,-0.745957,0.054781,-0.834653
3,-0.141420,1.119797,0.114574,-0.182811
4,2.523980,-1.542934,0.598557,-0.341996
5,1.514563,-0.987555,1.095007,0.001465
6,-1.358647,-1.088928,-0.643258,-0.118469
7,0.047709,-0.325359,-0.718633,-0.881978
8,3.013042,0.039229,-0.576829,-0.096285
9,1.639283,1.278942,-0.342460,1.076797


¡coincide con lo que calculamos anteriormente!

In [73]:
principalDf.describe().round(2)

,componente_principal_1,componente_principal_2,componente_principal_3,componente_principal_4
count,50.00,50.00,50.00,50.00
mean,-0.00,-0.00,-0.00,-0.00
std,1.59,1.00,0.60,0.42
min,-2.99,-1.57,-1.37,-0.95
25%,-1.12,-0.73,-0.43,-0.21
50%,-0.18,-0.15,0.03,-0.01
75%,1.37,0.77,0.26,0.21
max,3.01,2.39,2.04,1.08


Comprobamos que
<center>
$std(\sigma_{i}u_{i}) = \frac{\sigma_{i}}{\sqrt{n - 1}}$
</center>

In [74]:
print('std de la componente 1', S[0]/np.sqrt(A.shape[0]-1))  
print('std de la componente 2', S[1]/np.sqrt(A.shape[0]-1))
print('std de la componente 3', S[2]/np.sqrt(A.shape[0]-1))
print('std de la componente 4', S[3]/np.sqrt(A.shape[0]-1))

std de la componente 1 1.590867296236294
std de la componente 2 1.0049698708753338
std de la componente 3 0.6031914954510882
std de la componente 4 0.4206774028579882


# Gráfica de la primera componente

In [75]:
principalDf['cero_column'] = 0
fig = px.scatter(principalDf,
                 x='componente_principal_1', 
                 y='cero_column', 
                 color_discrete_sequence=['blueviolet'], 
                 opacity=0.4
                )
fig.update_xaxes(ticks='outside', 
                 tickwidth=1, 
                 tickcolor='gray', 
                 ticklen=5)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(
    scaleanchor = 'x',
    constrain='domain'
)
fig.update_layout(yaxis={'visible': False, 'showticklabels': False})
fig.update_traces(mode = 'markers', 
                  hovertemplate = None
)
fig.update_layout(hovermode='x unified')
fig.update_traces(
    hovertemplate =' ',
    showlegend = False,
)
fig.update_layout(
    hoverlabel = dict(
        bgcolor = 'LightSteelBlue',
        font_size = 40,
    )
)
fig.show()
#fig.write_image('componente_1.pdf', width=900, height=600)

# Gráfica de
<center>
$A_{1} = \sigma_{1}u_{1}v^T_{1}$
</center>

In [76]:
A_1 = pd.DataFrame()
A_1[['col_1']]= pd.DataFrame(Vt[0][0] * principalDf['componente_principal_1'])
A_1[['col_2']]= pd.DataFrame(Vt[0][1] * principalDf['componente_principal_1'])

In [77]:
import plotly.graph_objects as go
fig_1 = px.scatter(A_1, x="col_1", y="col_2",
                  opacity=0.3,
                   color_discrete_sequence=['blueviolet']
                  )
fig_2 = px.scatter(A_std, x=A_std.columns[0], y=A_std.columns[1], 
                    opacity=0.6,
                   color_discrete_sequence=['orange']
                   )
fig = go.Figure(data=fig_1.data + fig_2.data)
fig.show()
#fig.write_image("A_1.pdf", width=900, height=600)

# Notar que
<center>
$A = \sigma_{1}u_{1}v^T_{1} + \sigma_{2}u_{2}v^T_{2}$
</center>

In [78]:
principalDf['col_1'] = principalDf['componente_principal_2']  ## cambio de nombre de columnas para poder sumar
principalDf['col_2'] = principalDf['componente_principal_2'] 
A_2 = pd.DataFrame()
A_2[['col_1']]= A_1[['col_1']].add(pd.DataFrame(Vt[1][0] * principalDf['col_1']))
A_2[['col_2']]= A_1[['col_2']].add(pd.DataFrame(Vt[1][1] * principalDf['col_2']))
import plotly.graph_objects as go
fig_1 = px.scatter(A_2, x="col_1", y="col_2",
                  opacity=0.3,
                   color_discrete_sequence=['blueviolet']
                  )
fig_2 = px.scatter(A_std, x=A_std.columns[0], y=A_std.columns[1], 
                    opacity=0.3,
                   color_discrete_sequence=['orange']
                   )
fig = go.Figure(data=fig_1.data + fig_2.data)
fig.show()

# Varianza explicada y varianza total
Cada valor singular explica una parte de la varianza total:
<center>
$T = \frac{\sigma^2_{1} + \sigma^2_{2} + \sigma^2_{3} + \sigma^2_{4}}{n - 1}$
</center>
* El 1er valor singular explica la parte $\frac{\sigma^2_{1}}{n - 1}$ de $T$, el 2do valor singular explica la parte de $\frac{\sigma^2_{2}}{n - 1}$ de $T$

Y así sucesivamente si hubieran más valores singulares.

In [79]:
pca.explained_variance_

array([2.53085875, 1.00996444, 0.36383998, 0.17696948])

In [80]:
varianza_explicada_1 = (S[0]*S[0]/(A.shape[0]-1)).round(4)
varianza_explicada_2 = (S[1]*S[1]/(A.shape[0]-1)).round(4)
varianza_explicada_3 = (S[2]*S[2]/(A.shape[0]-1)).round(4)
varianza_explicada_4 = (S[3]*S[3]/(A.shape[0]-1)).round(4)
print('varianza_explicada_1 = ',varianza_explicada_1)
print('varianza_explicada_2 = ',varianza_explicada_2)
print('varianza_explicada_3 = ',varianza_explicada_3)
print('varianza_explicada_4 = ',varianza_explicada_4)

varianza_explicada_1 =  2.5309
varianza_explicada_2 =  1.01
varianza_explicada_3 =  0.3638
varianza_explicada_4 =  0.177


# Radio de varianza explicada
Para i = {1, 2, 3, 4}
* radio_varianza_explicada_i = $\frac{\sigma^2_{i}}{\sigma^2_{1} + \sigma^2_{2} + \sigma^2_{3} + \sigma^2_{4}}$

Usando la librería:

In [81]:
pca.explained_variance_ratio_ 

array([0.62006039, 0.24744129, 0.0891408 , 0.04335752])

Cálculo usando la fórmula:

In [82]:
radio_varianza_explicada_1 = (S[0]*S[0]/(S[0]*S[0]+S[1]*S[1]+S[2]*S[2]+S[3]*S[3])).round(4)
radio_varianza_explicada_2 = (S[1]*S[1]/(S[0]*S[0]+S[1]*S[1]+S[2]*S[2]+S[3]*S[3])).round(4)
radio_varianza_explicada_3 = (S[2]*S[2]/(S[0]*S[0]+S[1]*S[1]+S[2]*S[2]+S[3]*S[3])).round(4)
radio_varianza_explicada_4 = (S[3]*S[3]/(S[0]*S[0]+S[1]*S[1]+S[2]*S[2]+S[3]*S[3])).round(4)
print('radio_varianza_explicada_1 = ', radio_varianza_explicada_1)
print('radio-varianza_explicada_2 = ', radio_varianza_explicada_2)
print('radio_varianza_explicada_3 = ', radio_varianza_explicada_3)
print('radio-varianza_explicada_4 = ', radio_varianza_explicada_4)

radio_varianza_explicada_1 =  0.6201
radio-varianza_explicada_2 =  0.2474
radio_varianza_explicada_3 =  0.0891
radio-varianza_explicada_4 =  0.0434


**Conclusión:** Para este ejemplo podemos notar que la varianza observada en los datos puede ser explicada en su mayoría por los componentes 1 y 2, ya que estos explican la varianza de los datos en un 62% y 0.24% respectivamente.